# GenAI Usage of UC functions as Agent Tools

In this tutorial, we will be walking through the process of how to get started with using [Unity Catalog](https://www.unitycatalog.io/)'s function storage capabilities to serve as a GenAI Agent tool repository. 

**Prerequisites for this tutorial**:

1. A clone of the [Unity Catalog repository](https://github.com/unitycatalog/unitycatalog).

    ```sh
    git clone https://github.com/unitycatalog/unitycatalog
    ```

2. JDK-17 installed on your system (in order to build and run the Unity Catalog services)<sup>1</sup>
3. A Python installation version >= 3.9
4. [Docker Desktop](https://www.docker.com/products/docker-desktop/) (recommended; you can also install the docker engine yourself)

<sup>1</sup> (New to managing Java environments? [jenv](https://github.com/jenv/jenv) is a fantastic tool that can help!) 

**Package Requirements (Python)**:

You can install the required packages for this tutorial by running:

```shell
pip install unitycatalog-client unitycatalog-ai unitycatalog-openai
```

## Objectives

By the end of this tutorial you will have learned:

- How to start a Unity Catalog server using Docker
- How to use the `unitycatalog-client` to create a Catalog and a Schema to store your functions
- How to create Unity Catalog Functions to store Python callables
- How to list functions available in a given Catalog and Schema using the `unitycatalog-client` package
- How to execute stored functions for validation
- How to create a UCFunctionToolkit instance to register functions as tools for use in calls to OpenAI
- How to build a very simple tool calling Agent using OpenAI and Unity Catalog


## Starting a UC server

After you've cloned the Unity Catalog repository, navigate to the repository root (`unitycatalog`). Once there, simply run:

```shell
docker compose up
```

The Unity Catalog services will commence fetching, compiling, and building the server and UI infrastructure. Once complete, you will see in a terminal window:

```text
unitycatalog-server-1  | ###################################################################
unitycatalog-server-1  | #  _    _       _ _            _____      _        _              #
unitycatalog-server-1  | # | |  | |     (_) |          / ____|    | |      | |             #
unitycatalog-server-1  | # | |  | |_ __  _| |_ _   _  | |     __ _| |_ __ _| | ___   __ _  #
unitycatalog-server-1  | # | |  | | '_ \| | __| | | | | |    / _` | __/ _` | |/ _ \ / _` | #
unitycatalog-server-1  | # | |__| | | | | | |_| |_| | | |___| (_| | || (_| | | (_) | (_| | #
unitycatalog-server-1  | #  \____/|_| |_|_|\__|\__, |  \_____\__,_|\__\__,_|_|\___/ \__, | #
unitycatalog-server-1  | #                      __/ |                                __/ | #
unitycatalog-server-1  | #                     |___/               v0.3.0-SNAPSHOT  |___/  #
unitycatalog-server-1  | ###################################################################
unitycatalog-server-1  |
```

At this point, you can interface with the server as shown in this tutorial.

## Configuration

This next cell sets up access to your running UnityCatalog server, creates a Catalog and a Schema that we will be using throughout the remainder of this tutorial. 

> Note: The unitycatalog-client is an aiohttp-based package. When directly interfacing with the APIs in that package, make sure to use async interfaces when making calls. The `UnitycatalogFunctionClient` API offers synchronous (shown below) convenience methods for creating catalogs and schemas, though.

In [ ]:
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration

CATALOG = "AICatalogDemonstration"
SCHEMA = "AISchemaDemonstration"

config = Configuration(host="http://localhost:8080/api/2.1/unity-catalog")
client = ApiClient(configuration=config)

uc_client = UnitycatalogFunctionClient(api_client=client)

uc_client.uc.create_catalog(
    name=CATALOG, comment="A demonstration catalog for the AI functionality in Unity Catalog."
)
uc_client.uc.create_schema(
    name=SCHEMA,
    catalog_name=CATALOG,
    comment="A demonstration schema for holding tutorial Python functions for GenAI usage.",
)

SchemaInfo(name='AISchema', catalog_name='AICatalog', comment='This is a schema used for storing GenAI functions.', properties={}, full_name='AICatalog.AISchema', owner=None, created_at=1732297125510, created_by=None, updated_at=1732297125510, updated_by=None, schema_id='4925f53c-216a-44e7-9185-34c39fb9f51f')

## Create a Test function

Once we have our Catalog `"AICatalog"` and our Schema `"AISchema"` created, we can now register a test function to UnityCatalog. 

This simple test function is defined as any other Python function, with a few caveats that are required to be met in order for the function to be used as a GenAI tool. You must:

- **Define types**: type hints are **required** for all parameters and for the return type of the function.
- **Use a Docstring**: A Google-style Docstring is **required** for the `comment` block within UnityCatalog's function APIs to be populated and for the parameter description comments to be populated. Without a `comment`, your GenAI LLM will have no idea what the function is for or how to use it.
- **Local imports**: If you are using a library that is not defined within the core Python libraries, you should include your import within the function body.

> Note: If you would like to replace an existing function of the same name (your function name will be the name of your Python callable - in the case below, `"my_test_func"`), set `replace=True` to overwrite it.

In [2]:
def my_test_func(a: str, b: str) -> str:
    """
    Returns an upper case concatenation of two strings separated by a space.

    Args:
        a: the first string
        b: the second string

    Returns:
        Uppercased concatenation of the two strings.
    """
    # Concatenate the two strings with a space
    concatenated = f"{a} {b}"

    # Convert the concatenated string to uppercase
    uppercased = concatenated.upper()

    return uppercased


my_callable_func = uc_client.create_python_function(
    func=my_test_func,
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True,
)

my_callable_func

FunctionInfo(name='my_test_func', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='a', type_text='STRING', type_json='{"name": "a", "type": "string", "nullable": false, "metadata": {"comment": "the first string"}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='the first string'), FunctionParameterInfo(name='b', type_text='STRING', type_json='{"name": "b", "type": "string", "nullable": false, "metadata": {"comment": "the second string"}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=1, parameter_mode=None, parameter_type=None, parameter_default=None, comment='the second string')]), data_type=<ColumnTypeName.STRING: 'STRING'>, full_data_type='STRING', return_params=None, routine_body='EXTERNAL

## Verify function execution

Now that we have our function created within Unity Catalog, we can validate that the function is executable by using the `execute_function` API available within the `UnitycatalogFunctionClient` instance. 

> WARNING: Functions that are defined within Unity Catalog are executed in the **local environment that you are calling from**, within a subprocess. Be **very careful** when executing functions that you did not author and ensure that you understand the contents of the `route_definition` of any function that you are going to execute before doing so. You can inspect the contents of a function either by navigating to the Unity Catalog UI or by retrieving the `FunctionInfo` through the `get_function` API. 

In [3]:
FUNC = f"{CATALOG}.{SCHEMA}.my_test_func"

uc_client.execute_function(function_name=FUNC, parameters={"a": "hi", "b": "there"})

FunctionExecutionResult(error=None, format='SCALAR', value='HI THERE', truncated=None)

## Create the functions for our Agent

Now that we've seen how to create and verify the functionality of a simple test function, let's define some functions that we'll be creating for our OpenAI Agent. 

What this simple tool calling Agent will do is provide an interface for messaging a weather forecast 

In [4]:
def fahrenheit_to_celsius(fahrenheit: float) -> float:
    """
    Converts temperature from Fahrenheit to Celsius.

    Args:
        fahrenheit (float): Temperature in degrees Fahrenheit.

    Returns:
        float: Temperature in degrees Celsius.
    """
    return (fahrenheit - 32) * 5.0 / 9.0


def calculate_humidex_temperature(temperature_c: float, humidity: float) -> float:
    """
    Calculates the Humidex temperature based on the actual temperature in Celsius and relative humidity.
    High temperatures with high humidity feel hotter, while low temperatures with low humidity feel colder.
    This function uses the Humidex formula to compute the perceived temperature.

    Args:
        temperature_c (float): Actual temperature in degrees Celsius.
        humidity (float): Relative humidity percentage (0-100).

    Returns:
        float: Real feel temperature in degrees Celsius.
    """
    import math

    if humidity < 0 or humidity > 100:
        raise ValueError("Humidity must be between 0 and 100 percent.")

    # Calculate water vapor pressure (e) in millibars
    e = 6.11 * math.exp(5417.7530 * ((1 / 273.16) - (1 / (temperature_c + 273.15))))

    # Humidex formula
    return temperature_c + 0.5555 * (e * humidity / 100 - 10)

In [5]:
# Create a UC function for Fahrenheit to Celsius conversion
func1 = uc_client.create_python_function(
    func=fahrenheit_to_celsius,
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True,
)

# Create a UC function based on the Humidex callable
func2 = uc_client.create_python_function(
    func=calculate_humidex_temperature,
    catalog=CATALOG,
    schema=SCHEMA,
    replace=True,
)

### Verify function creation

Now that we've created our Python functions `fahrenheit_to_celsius` and `calculate_humidex_temperature` within our schema that we've created, we can use the `UnitycatalogFunctionClient` to list all of the functions that are contained within a given schema. 

In [6]:
names = [info.name for info in uc_client.list_functions(catalog=CATALOG, schema=SCHEMA)]

names

['calculate_humidex_temperature', 'fahrenheit_to_celsius', 'my_test_func']

## Run our functions

When executing a function, there are two inputs required in the caller's interface:

- **function_name**: The fully qualified name in the form `[catalog].[schema].[function name]`.
    Since we used the `create_python_function` API, the *function name* is the name of our callable that we passed in.
- **parameters**: A dictionary input (passed effectively as a `**kwargs` input to our function). The keys that are required for the parameters dictionary are the arguments to our function. 

In [7]:
# Run the functions to ensure that everything is working properly
f_to_c = f"{CATALOG}.{SCHEMA}.fahrenheit_to_celsius"

celsius = uc_client.execute_function(function_name=f_to_c, parameters={"fahrenheit": 88.3})

celsius

FunctionExecutionResult(error=None, format='SCALAR', value='31.27777777777778', truncated=None)

In [8]:
humidex_func = f"{CATALOG}.{SCHEMA}.calculate_humidex_temperature"

humidex = uc_client.execute_function(
    function_name=humidex_func, parameters={"temperature_c": float(celsius.value), "humidity": 72.6}
)

humidex

FunctionExecutionResult(error=None, format='SCALAR', value='44.61870689130167', truncated=None)

## Build a tool calling Agent with OpenAI

In order to let a GenAI service like a GPT model hosted by OpenAI use our functions, we need to register them as tools. 

To do this, we'll import the Unity Catalog AI OpenAI integration package and utilize the `UCFunctionToolkit` class to construct the interface we need to register tools. 

> Note: Each integration for UnityCatalog's AI libraries utilizes the same interface name `UCFunctionToolkit`. If you need to create a complex workflow that involves integrations between different services, ensure that you alias your imports for the toolkit constructors.

In [9]:
from unitycatalog.ai.openai.toolkit import UCFunctionToolkit

In [10]:
# Create the toolkit instance with the fully qualified function names that we defined earlier.
toolkit = UCFunctionToolkit(client=uc_client, function_names=[func1.full_name, func2.full_name])

# Extract the `tools` property of the toolkit so that the tool definitions can be passed to the OpenAI LLM.
tools = toolkit.tools

tools

[{'type': 'function',
  'function': {'name': 'AICatalog__AISchema__fahrenheit_to_celsius',
   'strict': True,
   'parameters': {'properties': {'fahrenheit': {'description': 'Temperature in degrees Fahrenheit.',
      'title': 'Fahrenheit',
      'type': 'number'}},
    'title': 'AICatalog__AISchema__fahrenheit_to_celsius__params',
    'type': 'object',
    'additionalProperties': False,
    'required': ['fahrenheit']},
   'description': 'Converts temperature from Fahrenheit to Celsius.'}},
 {'type': 'function',
  'function': {'name': 'AICatalog__AISchema__calculate_humidex_temperature',
   'strict': True,
   'parameters': {'properties': {'temperature_c': {'description': 'Actual temperature in degrees Celsius.',
      'title': 'Temperature C',
      'type': 'number'},
     'humidity': {'description': 'Relative humidity percentage (0-100).',
      'title': 'Humidity',
      'type': 'number'}},
    'title': 'AICatalog__AISchema__calculate_humidex_temperature__params',
    'type': 'object'

#### Verify that we can communicate with OpenAI

In [11]:
# Ensure that the OpenAI API Key is set within the environment
import os

assert os.environ.get("OPENAI_API_KEY")

## Interface with OpenAI's GPT model with tool calling capabilities

In the following code block, we submit our messages to OpenAI. As with a standard query, we provide both a system prompt message and a user message. 

In order to allow the LLM to utilize tool calling capabilities, we pass our toolkit definitions (`toolkit.tools`) to the `tools` argument within the OpenAI SDK `chat.completions.create` API. With the tool definitions provided, OpenAI's LLM is available to contextually 'decide' when it is appropriate to request a tool call to be executed with its response. 

In the case of the question that we're providing, OpenAI will recognize that it should call both of our tools in order to facilitate an accurate answer.

In [12]:
import openai

initial_messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that is designed to provide recommendations to help keep users comforable and "
        "safe with respect to weather forecast questions. Please suggestions on what to wear, what additional context a user "
        "might need for outdoor adventures.",
    },
    {
        "role": "user",
        "content": "The forecast tomorrow is 97.3F with 80.6% humidity. Should I go on a hike in the mountains?",
    },
]

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=initial_messages,
    tools=tools,
)

# The `finish_reason` for this response is `tool_calls` indicating that the LLM is requesting the response from executing the tools we
# defined for that purpose.
response

ChatCompletion(id='chatcmpl-AWTOmZx4lJvMMnegSC9J4DKh7SCyQ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8Fi4ufZL2sA2zC5fr5Af7EMO', function=Function(arguments='{"fahrenheit": 97.3}', name='AICatalog__AISchema__fahrenheit_to_celsius'), type='function'), ChatCompletionMessageToolCall(id='call_GxRGWyzCtTIXnzEp8MMRGx6w', function=Function(arguments='{"temperature_c": 36.333333333333336, "humidity": 80.6}', name='AICatalog__AISchema__calculate_humidex_temperature'), type='function')]))], created=1732302868, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0705bf87c0', usage=CompletionUsage(completion_tokens=81, prompt_tokens=270, total_tokens=351, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0,

### Generate the tool call messages

With the response from OpenAI, we can then use a utility function in `unitycatalog-openai` to help format, call our function, and create the appropriate response format for the response call back to OpenAI. 

In [13]:
from unitycatalog.ai.openai.utils import generate_tool_call_messages

messages = generate_tool_call_messages(response=response, client=uc_client)

messages

[{'content': None,
  'refusal': None,
  'role': 'assistant',
  'tool_calls': [{'id': 'call_8Fi4ufZL2sA2zC5fr5Af7EMO',
    'function': {'arguments': '{"fahrenheit": 97.3}',
     'name': 'AICatalog__AISchema__fahrenheit_to_celsius'},
    'type': 'function'},
   {'id': 'call_GxRGWyzCtTIXnzEp8MMRGx6w',
    'function': {'arguments': '{"temperature_c": 36.333333333333336, "humidity": 80.6}',
     'name': 'AICatalog__AISchema__calculate_humidex_temperature'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '{"content": "36.27777777777778"}',
  'tool_call_id': 'call_8Fi4ufZL2sA2zC5fr5Af7EMO'},
 {'role': 'tool',
  'content': '{"content": "58.83411235106172"}',
  'tool_call_id': 'call_GxRGWyzCtTIXnzEp8MMRGx6w'}]

### Pass the results back to OpenAI

Now that we have the results from our tool call executions (both of them), generated in the format needed to make the next response call, we can submit this payload back to OpenAI so that the LLM can respond with their final recommendations! 

> NOTE: In order for OpenAI's LLM services to have the full context of the conversation history (the APIs are stateless), the history of the session's messages should be prepended to each additional message (as shown below with `final_messages = initial_messages + messages`).

In [14]:
# Prepend the message history
final_messages = initial_messages + messages

response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=final_messages,
    tools=tools,
)

response

ChatCompletion(id='chatcmpl-AWTOoKvQA9AIRfrKjBuoM5ypbheU7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The temperature is approximately 36.3°C (97.3°F), and with a high humidity of 80.6%, the humidex (perceived temperature) feels like about 58.8°C (137.8°F). This is extremely high and can be dangerous for outdoor activities like hiking.\n\n**Recommendations:**\n- **Clothing:** Wear lightweight, breathable, and moisture-wicking clothing to help with sweat evaporation. A wide-brimmed hat and sunglasses are also important for sun protection.\n- **Hydration:** Carry plenty of water to stay hydrated. The heat and humidity can lead to dehydration quickly.\n- **Timing:** If you decide to hike, consider going very early in the morning or later in the evening when temperatures are cooler.\n- **Pace:** Take breaks often, and listen to your body. If you feel dizzy, weak, or excessively tired, it's best to stop and seek shade or a cool envi

In [15]:
response.choices[0].message.content

"The temperature is approximately 36.3°C (97.3°F), and with a high humidity of 80.6%, the humidex (perceived temperature) feels like about 58.8°C (137.8°F). This is extremely high and can be dangerous for outdoor activities like hiking.\n\n**Recommendations:**\n- **Clothing:** Wear lightweight, breathable, and moisture-wicking clothing to help with sweat evaporation. A wide-brimmed hat and sunglasses are also important for sun protection.\n- **Hydration:** Carry plenty of water to stay hydrated. The heat and humidity can lead to dehydration quickly.\n- **Timing:** If you decide to hike, consider going very early in the morning or later in the evening when temperatures are cooler.\n- **Pace:** Take breaks often, and listen to your body. If you feel dizzy, weak, or excessively tired, it's best to stop and seek shade or a cool environment.\n- **Safety:** Be aware of the signs of heat exhaustion and heat stroke.\n\nGiven the extreme weather conditions, it may be safer to postpone your hike